# Clasificación de tokens con HuggingFace

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../datos/procesamiento')
from corpus import Corpus
from etiquetado_entidades import codigo_hfl, split_rule, process_column

Cargando datos

In [2]:
main_db = pd.read_csv('../datos/DATA_HLF_MDS_2.csv',sep=',')

code_db = pd.read_excel('../datos/PRINCIPIOS_ACTIVOS_MDS.xlsx')
HLF = code_db.loc[:,['PRINCIPIO_ACTIVO','CODIGO_HLF']]
HLF_df = codigo_hfl(HLF)

df = main_db.join(HLF_df.set_index('CODIGO_MEDICAMENTO'), on='CODIGO_MEDICAMENTO')

Cargar principios activos, forma farma y juntar columnas

In [3]:
#PARA DETECTAR LA POSICION DE ENTIDADES TIPO PRINCIPIO ACTIVO, FORMA FARMACO SE COMPARARAN LOS ELEMENTOS DEL CORPUS CON LAS LISTAS CORRESPONDIENTES.
PA = np.unique(split_rule(df['PRINCIPIO_ACTIVO'].dropna().unique()))
FF = np.unique(split_rule(df['FORMA_FARMA'].dropna().unique()))

rows = (df['PRES_DENOMINACION'] + ' ' + df['RESUMEN']).dropna().unique()

Ejecutar procesamiento de datos de Martin

In [4]:
corpus = Corpus()

for i in range(len(rows)):
    row = rows[i].split()
    

    # begin patch
    err_count = 0
    try:
        tagged_seq = process_column(row,PA,FF)
        corpus.append(tagged_seq)
    except ValueError:
        print("Advertencia: error en etiquetado")
        print(tagged_seq.tokens)
        print(tagged_seq.tags)
        err_count += 1

    if i > 100000 + err_count:  # end patch
        # if i > 100:
        # mismo tamaño que baseline  martin
        break

Advertencia: error en etiquetado
['PREDNISONA', '20', 'MG', 'COMPRIMIDO', '40', 'MG', 'ORAL', 'cada', '24', 'horas', 'durante', '4', 'dias']
['B-ACTVPRNCP', 'O', 'O', 'B-ADMIN', 'O', 'O', 'B-ADMIN', 'B-PERIODICITY', 'I-PERIODICITY', 'I-PERIODICITY', 'B-DURATION', 'I-DURATION', 'I-DURATION']


---

## Cargando el corpus como dataset de HuggingFace

In [5]:
ner_dict = {'O': 0,
            'B-ACTVPRNCP': 1,
            'I-ACTVPRNCP': 2,
            'B-ADMIN': 3,
            'I-ADMIN': 4,
            'B-PERIODICITY': 5,
            'I-PERIODICITY': 6,
            'B-DURATION': 7,
            'I-DURATION': 8
            }

corpus.entidades = ner_dict

In [6]:
HF_dataset = corpus.to_HF_dataset()

In [7]:
HF_dataset.data

InMemoryTable
id: int64
tokens: list<item: string>
  child 0, item: string
ner_tags: list<item: int64>
  child 0, item: int64
----
id: [[0,1,2,3,4,...,99996,99997,99998,99999,100000]]
tokens: [[["PARACETAMOL","500","MG","COMPRIMIDO","1",...,"6","horas","durante","3","dias"],["KETOROLACO","10","MG","COMPRIMIDO","1",...,"8","horas","durante","3","dias"],...,["DEXAMETASONA","4","MG/ML","SOLUCIÓN","INYECTABLE",...,"UNIDAD","PARENTERAL","cada","8","horas"],["KETOROLACO","TROMETAMOL","30","MG/ML","SOLUCIÓN",...,"MG","INTRAVENOSA","EN","BOLO","diaria"]]]
ner_tags: [[[1,0,0,3,0,...,6,6,7,8,8],[1,0,0,3,0,...,6,6,7,8,8],...,[1,0,0,1,2,...,0,0,5,6,6],[1,0,0,0,1,...,0,0,3,0,0]]]

In [8]:
HF_dataset.features[f"ner_tags"].feature.names = [key for key in ner_dict.keys()]
HF_dataset.features[f"ner_tags"].feature.names

['O',
 'B-ACTVPRNCP',
 'I-ACTVPRNCP',
 'B-ADMIN',
 'I-ADMIN',
 'B-PERIODICITY',
 'I-PERIODICITY',
 'B-DURATION',
 'I-DURATION']

In [9]:
HF_dataset = HF_dataset.train_test_split(test_size=0.2,seed=0)
HF_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 80000
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20001
    })
})

## Cargamos un modelo basado en Transformers de HuggingFace

Nuestro modelo será: plncmm/bert-clinical-scratch-wl-es. Ha sido fine-tuneado con texto médico (aunque probablemente no con prescripciones)

[Fuente de esta sección](https://huggingface.co/docs/transformers/tasks/token_classification)

### Tokenizer

In [10]:
MODEL = "plncmm/bert-clinical-scratch-wl-es"

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)

Usaremos el tokenizador para codificar nuestro input.

In [12]:
example = HF_dataset["train"][0]

tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'aci',
 '##do',
 'tran',
 '##ex',
 '##ami',
 '##co',
 '1',
 '##g',
 '/',
 '10',
 '##ml',
 'inyec',
 '##y',
 '##table',
 '2',
 'unidad',
 'intraven',
 '##osa',
 'cada',
 '2',
 '##4',
 'horas',
 '[SEP]']

Debemos corregir el _mismatch_ entre input tokenizado y la lista de tokens

In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
tokenized_data = HF_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_data = tokenized_data.remove_columns(['id','tokens','ner_tags'])

  0%|          | 0/80 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

In [15]:
tokenized_data.data

{'train': InMemoryTable
 input_ids: list<item: int32>
   child 0, item: int32
 token_type_ids: list<item: int8>
   child 0, item: int8
 attention_mask: list<item: int8>
   child 0, item: int8
 labels: list<item: int64>
   child 0, item: int64
 ----
 input_ids: [[[4,15721,1050,1572,2483,...,1748,1129,1003,2596,5],[4,15772,13955,3284,16056,...,1976,1748,997,2596,5],...,[4,1097,23070,14921,3284,...,2427,2242,1074,1707,5],[4,19051,25612,1492,25579,...,1411,2242,1074,1707,5]],[[4,15772,13955,3284,16056,...,15359,1672,1972,12873,5],[4,19769,2470,19624,16057,...,1748,1129,1003,2596,5],...,[4,5536,8413,1213,30962,...,1748,1129,1003,2596,5],[4,6752,5899,30957,4129,...,28982,1976,9934,2940,5]],...,[[4,5647,1699,23565,15709,...,2596,1672,999,12873,5],[4,19890,1577,1343,2633,...,12791,1748,1992,2596,5],...,[4,1067,7785,1197,3092,...,1707,1672,2286,12873,5],[4,6265,1446,1600,30962,...,1748,1129,1003,2596,5]],[[4,5536,8413,1213,30962,...,1976,1748,1413,2885,5],[4,6265,1446,1600,30962,...,1976,1748,9

### Modelo

In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(MODEL, num_labels=len(ner_dict))

Some weights of the model checkpoint at plncmm/bert-clinical-scratch-wl-es were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at plncmm/bert-clinical-s

## Entrenamiento

In [17]:
training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    weight_decay = 0.01,
)

In [18]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_data["train"],
    eval_dataset = tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator = data_collator,
)

In [19]:
trainer.train()

/home/camilo/miniconda3/envs/entidades_env/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 80000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 25000


  0%|          | 0/25000 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json


{'loss': 0.164, 'learning_rate': 1.9600000000000002e-05, 'epoch': 0.1}


Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json


...

Model weights saved in ./results/checkpoint-25000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-25000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-25000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 20001
  Batch size = 16


  0%|          | 0/1251 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.003244617022573948, 'eval_runtime': 58.6449, 'eval_samples_per_second': 341.053, 'eval_steps_per_second': 21.332, 'epoch': 5.0}
{'train_runtime': 4891.2761, 'train_samples_per_second': 81.778, 'train_steps_per_second': 5.111, 'train_loss': 0.008898326160907745, 'epoch': 5.0}


TrainOutput(global_step=25000, training_loss=0.008898326160907745, metrics={'train_runtime': 4891.2761, 'train_samples_per_second': 81.778, 'train_steps_per_second': 5.111, 'train_loss': 0.008898326160907745, 'epoch': 5.0})

In [20]:
trainer.save_model("bert-clinical-scratch-wl-es-NER-prescription")

Saving model checkpoint to bert-clinical-scratch-wl-es-NER-prescription
Configuration saved in bert-clinical-scratch-wl-es-NER-prescription/config.json
Model weights saved in bert-clinical-scratch-wl-es-NER-prescription/pytorch_model.bin
tokenizer config file saved in bert-clinical-scratch-wl-es-NER-prescription/tokenizer_config.json
Special tokens file saved in bert-clinical-scratch-wl-es-NER-prescription/special_tokens_map.json


## Evaluación de resultados

In [21]:
model = AutoModelForTokenClassification.from_pretrained("bert-clinical-scratch-wl-es-NER-prescription")

loading configuration file bert-clinical-scratch-wl-es-NER-prescription/config.json
Model config BertConfig {
  "_name_or_path": "bert-clinical-scratch-wl-es-NER-prescription",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "

In [35]:
from scipy.special import softmax

def eval_text(text):
    encoded_input = tokenizer(text,return_tensors='pt',is_split_into_words=isinstance(text,list))
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return np.argmax(scores,axis=1)

In [36]:
text = "PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias"

eval_text(text)

array([6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [30]:
def map_entities(y_pred,map_dict):
    inv_map = {v: k for k, v in map_dict.items()}
    return np.array([inv_map[y] for y in y_pred])

In [31]:
map_entities(eval_text(text),ner_dict)

array(['I-PERIODICITY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',
       'O', 'O', 'O', 'O', 'O', 'O'], dtype='<U13')

In [43]:
y_test = [row['ner_tags'] for row in HF_dataset['test']]

In [44]:
y_preds = [list(eval_text(row['tokens'])) for row in HF_dataset['test']]

In [42]:
from seqeval.metrics import f1_score, precision_score, recall_score

def calculate_metrics(y_pred, y_true, print=True):
    """
    Calcula precision, recall y f1
    """
    # calcular scores
    f1 = f1_score(y_true, y_pred, mode='strict')
    precision = precision_score(y_true, y_pred, mode='strict')
    recall = recall_score(y_true, y_pred, mode='strict')

    if print:
        print("Resultados de evaluación")
        print(f'\t f1: {f1:.2f} | precision: {precision:.2f} | recall: {recall:.2f}')

    return precision, recall, f1

In [45]:
calculate_metrics(y_preds,y_test)

ValueError: Found input variables with inconsistent numbers of samples:
[14, 12, 15, 13, 15, 16, 11, 17, 17, 15, 12, 10, 10, 17, 14, 15, 8, 14, 16, 24, 15]
[24, 17, 29, 19, 27, 28, 15, 32, 28, 27, 19, 14, 16, 31, 19, 25, 13, 25, 24, 47, 30]